In [7]:
from flask import Flask, request, jsonify, render_template_string
from tensorflow.keras.preprocessing import image
import numpy as np
import tensorflow as tf
import os

app = Flask(__name__)

# Load the trained model
model_path = os.path.abspath('image_model.h5')
model = tf.keras.models.load_model(model_path)


# Image input size for CIFAR-10
IMG_SIZE = 32

# Ensure uploads folder exists
if not os.path.exists('uploads'):
    os.makedirs('uploads')

# Preprocess image
def prepare_image(img_path):
    img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = image.img_to_array(img) / 255.0
    return np.expand_dims(img_array, axis=0)

# HTML form for image upload
@app.route('/')
def index():
    return render_template_string("""
    <!DOCTYPE html>
    <html>
    <head><title>Image Classification</title></head>
    <body>
        <h1>Upload an Image for Classification</h1>
        <form action="/predict" method="POST" enctype="multipart/form-data">
            <input type="file" name="file" accept="image/*" required>
            <button type="submit">Classify</button>
        </form>
    </body>
    </html>
    """)

# Prediction route
@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'})

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'})

    if file:
        file_path = os.path.join('uploads', file.filename)
        try:
            file.save(file_path)
        except Exception as e:
            return jsonify({'error': f'Error saving file: {str(e)}'})

        if not file.filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            return jsonify({'error': 'Invalid image file'})

        try:
            img = prepare_image(file_path)
            prediction = model.predict(img)
            predicted_class = np.argmax(prediction, axis=1)[0]

            # CIFAR-10 labels
            if predicted_class == 3:
                label = 'Cat'
            elif predicted_class == 5:
                label = 'Dog'
            else:
                label = 'Not a Cat or Dog'

            return jsonify({'prediction': label})

        except Exception as e:
            return jsonify({'error': f'Prediction error: {str(e)}'})

# Start Flask app
app.run(host='0.0.0.0', port=5000, debug=False)



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [08/Apr/2025 15:24:31] "GET / HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step


127.0.0.1 - - [08/Apr/2025 15:24:37] "POST /predict HTTP/1.1" 200 -
Exception in thread Thread-3:
Traceback (most recent call last):
  File "C:\Users\SohailChougle\anaconda3\envs\Chatbot\Lib\site-packages\urllib3\connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\SohailChougle\anaconda3\envs\Chatbot\Lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    raise err
  File "C:\Users\SohailChougle\anaconda3\envs\Chatbot\Lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\SohailChougle\anaconda3\envs\Chatbot\Lib\site-packages\urllib3\connectionpool.py", line 787, in urlopen
    response = self._make_